In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
dataset = pd.read_csv('noah.csv')
dataset.head()

,feedline_width_fw_mm,patch_length_pl_mm,patch_width_pw_mm,resonant_frequency_ghz,minimum_s11_db,bandwidth_ghz,z_real,z_imaginary
0,0.4,2.45,2.95,39.289,-25.766,1.365000,62.973554,1.428412
1,0.4,2.45,2.99,39.232,-30.086,1.368000,65.557081,0.994853
2,0.4,2.45,3.03,39.175,-37.281,1.359001,67.832519,0.409622
3,0.4,2.45,3.07,39.124,-44.140,1.347000,70.517160,-0.187324
4,0.4,2.45,3.11,39.070,-32.343,1.335003,73.002861,-0.859542


In [3]:
x = dataset.iloc[:, 3:6]
y = dataset.iloc[:, 6:]

In [4]:
x.head()

,resonant_frequency_ghz,minimum_s11_db,bandwidth_ghz
0,39.289,-25.766,1.365000
1,39.232,-30.086,1.368000
2,39.175,-37.281,1.359001
3,39.124,-44.140,1.347000
4,39.070,-32.343,1.335003


In [5]:
y.head()

,z_real,z_imaginary
0,62.973554,1.428412
1,65.557081,0.994853
2,67.832519,0.409622
3,70.517160,-0.187324
4,73.002861,-0.859542


# Run this from now on

In [6]:

from catboost import CatBoostRegressor
from sklearn.multioutput import MultiOutputRegressor

# Set the best parameters
best_params = {
    "subsample": 0.6,
    "learning_rate": 0.01,
    "l2_leaf_reg": 0.1,
    "iterations": 1000,
    "depth": 6,
    "colsample_bylevel": 0.8,
    "objective": "RMSE",
    "random_state": 42,
    "silent": True,
    "train_dir": "./catboost"
}

base_model = CatBoostRegressor(**best_params)
multi_output_model = MultiOutputRegressor(base_model)

# Fit the model
multi_output_model.fit(x, y)


MultiOutputRegressor(estimator=<catboost.core.CatBoostRegressor object at 0x000002774733E6D0>)

In [7]:
import joblib
joblib.dump(multi_output_model, "genesis_wing.pkl")

['genesis_wing.pkl']

In [8]:
model = joblib.load("genesis_wing.pkl")

In [9]:
def predict_patch_dimensions(model, freq, s11, bandwidth):
    input_data = pd.DataFrame([[freq, s11, bandwidth]], columns=["resonant_frequency_ghz", "minimum_s11_db", "bandwidth_ghz"])
    prediction = model.predict(input_data)
    return {
        "z_real": prediction[0, 0],
        "z_imagenary": prediction[0, 1]
    }


In [12]:
result = predict_patch_dimensions(model, 39.232, -30.989, 1.3)
print(result)

{'z_real': np.float64(58.336322702408054), 'z_imagenary': np.float64(-1.0824010699489404)}
